In [34]:
from Bio import SeqIO
from io import StringIO
from gzip import open as gzopen
from Bio.SeqIO.FastaIO import SimpleFastaParser
import numpy as np
import matplotlib.pyplot as plt
import os 
import pandas as pd

pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [35]:
file = "/mnt/c/works/RKI/AMPsConverter/AMP_DB/stats/nr100"
lengths = map(len, SeqIO.parse(file, 'fasta'))
df = pd.Series([x for x in lengths], name="length").to_frame()

In [36]:
df.length.describe()

count    21226.000000
mean        60.496938
std        114.696874
min         11.000000
25%         18.000000
50%         28.000000
75%         61.000000
max       2337.000000
Name: length, dtype: float64

In [37]:
df['length'].value_counts(bins=100)

(8.673, 34.26]        12220
(34.26, 57.52]         3384
(57.52, 80.78]         2620
(80.78, 104.04]        1208
(127.3, 150.56]         218
                      ...  
(1592.68, 1615.94]        0
(1639.2, 1662.46]         0
(1662.46, 1685.72]        0
(1708.98, 1732.24]        0
(1150.74, 1174.0]         0
Name: length, Length: 100, dtype: int64

In [38]:
df_count=df.groupby('length').length.count().reset_index(name='count').sort_values(['count'], ascending=False)
df_count.to_csv('df_count.csv', index=False)
_tmp_df_count =df_count
_tmp_df_count

,length,count
9,20,1131
2,13,1119
1,12,912
7,18,873
4,15,855
...,...,...
392,448,1
394,455,1
396,460,1
397,461,1


In [39]:
from Bio.SeqIO.FastaIO import SimpleFastaParser

with open("/mnt/c/works/RKI/uniref50.len2400.fasta") as record_len2400:
    for name, seq in SimpleFastaParser(record_len2400):
        print(name)

67HNJ0_CALVF
UniRef50_A0A0A9YTV5 BHLH domain-containing protein n=3 Tax=Mirini TaxID=236659 RepID=A0A0A9YTV5_LYGHE
UniRef50_A0A090L570 Epidermal growth factor-like domain and Insulin-like growth factor binding protein, N-terminal domain-containing protein n=1 Tax=Strongyloides ratti TaxID=34506 RepID=A0A090L570_STRRB
UniRef50_A0A1S3J519 Chitin synthase n=1 Tax=Lingula unguis TaxID=7574 RepID=A0A1S3J519_LINUN
UniRef50_A0A1S3J880 uncharacterized protein LOC106170922 n=1 Tax=Lingula unguis TaxID=7574 RepID=A0A1S3J880_LINUN
UniRef50_A0A3S1BG16 Cnn_1N domain-containing protein n=1 Tax=Elysia chlorotica TaxID=188477 RepID=A0A3S1BG16_ELYCH
UniRef50_A0A1J1HC04 GYF domain-containing protein n=2 Tax=Plasmodium (Haemamoeba) TaxID=418104 RepID=A0A1J1HC04_PLARL
UniRef50_A0A2G8L7T2 ALMS_motif domain-containing protein n=1 Tax=Stichopus japonicus TaxID=307972 RepID=A0A2G8L7T2_STIJA
UniRef50_A2EN90 Beige/BEACH domain containing protein n=1 Tax=Trichomonas vaginalis TaxID=5722 RepID=A2EN90_TRIVA
UniRef

KeyboardInterrupt: 

In [30]:
_tmp_df_count.loc[ _tmp_df_count['length'] == 20]['count'].values +1

9    1128
Name: count, dtype: int64

In [47]:
_tmp_df_count['count'].values.sum() == 0

False

In [49]:
LenFile = open('non_AMPs.csv', 'w')

with open("/mnt/c/works/RKI/uniref50.len2400.fasta") as record_len2400:
      for _name, _seq in  SimpleFastaParser(record_len2400):
           seqLen = len(_seq)
           #update value
           _tmp_row = _tmp_df_count.loc[ _tmp_df_count['length'] == seqLen]
           if not _tmp_row.empty and  _tmp_row['count'].values > 0:
               #print (str(_tmp_row['length']) +" "+ str(_tmp_row['count']))
               LenFile.write(_name+','+_seq+',' + str(seqLen) + '\n')
               _tmp_df_count.at[ _tmp_row.index,'count']= _tmp_row['count'] - 1
           else:
               if _tmp_df_count['count'].values.sum() == 0:
                   print("stop")
                   break

        


In [8]:
LenFile = open('non_AMPs.csv', 'w')

with open("/mnt/c/works/RKI/uniref50.len2400.fasta") as record_len2400:
    for index, row in df_count.iterrows():
        _count= row['count']
        _length= row['length']
        print("_length:"+ str(_length))
        for _name, _seq in  SimpleFastaParser(record_len2400):
            seqLen = len(_seq)
            if _length == seqLen:
                #print("seqLen:"+ str(seqLen))
                LenFile.write(_name+','+_seq+',' + str(seqLen) + '\n')
                _count = _count - 1 
            if _count == 0:
                break;
        
        if _count != 0:
            print("Remaining:"+ str(_count) +" " + str(_length))


967
seqLen:1967
seqLen:1945
seqLen:1700
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1731
seqLen:1821
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1748
seqLen:1967
seqLen:1967
seqLen:1942
seqLen:1967
seqLen:1925
seqLen:1900
seqLen:1967
seqLen:1806
seqLen:1967
seqLen:1960
seqLen:1967
seqLen:23
seqLen:1780
seqLen:1924
seqLen:1967
seqLen:1967
seqLen:1837
seqLen:1967
seqLen:1948
seqLen:1967
seqLen:1965
seqLen:242
seqLen:1762
seqLen:1932
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1914
seqLen:1967
seqLen:1967
seqLen:1875
seqLen:1967
seqLen:1967
seqLen:1967
seqLen:1962
seqLen:1967
seqLen:1955
seqLen:1581
seqLen:1908
seqLen:1748
seqLen:1838
seqLen:1601
seqLen:1705
seqLen:1950
seqLen:1925
seqLen:1967
seqLen:1967
seqLen:1541
seqLen:1925
seqLen:1967
seqLen:1952
seqLen:1967
seq

KeyboardInterrupt: 

In [ ]:
def batch_iterator(iterator, batch_size):
    """Returns lists of length batch_size.

    This can be used on any iterator, for example to batch up
    SeqRecord objects from Bio.SeqIO.parse(...), or to batch
    Alignment objects from Bio.AlignIO.parse(...), or simply
    lines from a file handle.

    This is a generator function, and it returns lists of the
    entries from the supplied iterator.  Each list will have
    batch_size entries, although the final list may be shorter.
    """
    entry = True  # Make sure we loop once
    while entry:
        batch = []
        while len(batch) < batch_size:
            try:
                entry =  next(iterator)
            except StopIteration:
                entry = None
            if entry is None:
                # End of file
                break
            batch.append(entry)
        if batch:
            yield batch
            
record_iter = SeqIO.parse(gzopen("uniref50.fasta.gz", "rt"), "fasta")
for i, batch in enumerate(batch_iterator(record_iter, 200000)):
    filename = "group_%i.fasta" % (i + 1)
    with open(filename, "w") as handle:
        count = SeqIO.write(batch, handle, "fasta")
    print("Wrote %i records to %s" % (count, filename))